In [7]:
import os
os.sys.path.insert(0, '../script')
from webnlg import WebNLGCorpus
import re

In [219]:
v_12 = WebNLGCorpus.load(['train_1.2', 'dev_1.2'])

In [81]:
from functools import partial

class Replacer:
    
    def __init__(self):
        self._i = -1
        
    def replace_sop(self, m):
        
        self._i += 1
        
        return "(?P<{}_{}>.*?)".format(next((v for v in m.groupdict().values() if v)), self._i)

In [213]:
replace_hifen_w_underline = re.compile(r'(?<=((RIDGE)|(TIENT)|(AGENT)))(\-)')
make_groups_re = re.compile(r'((?P<bridge>BRIDGE_\d)|(?P<patient>PATIENT_\d)|(?P<agent>AGENT_\d))')
REMOVE_SPACE_BEFORE_DOT_RE = re.compile(r"(.*?)(\s*)([\.,'!])")

def remove_space_before_dot(s):
    
    return REMOVE_SPACE_BEFORE_DOT_RE.sub(r'\1\3', s)

transtab = str.maketrans("’'", "''")
def normalize_text(s):
    
    return s.translate(transtab).replace('`` ', '').replace(" '' ", '').replace('--', '-')

def extract(text, template, entity_map):
    
    template = normalize_text(template)
    template = replace_hifen_w_underline.sub(r'_', template)
    template = remove_space_before_dot(template)
    
    r = Replacer()
    template_re = re.compile('^{}$'.format(make_groups_re.sub(r.replace_sop, re.escape(template))))
    
    text = remove_space_before_dot(text)
    
    m = template_re.match(text)
    
    if m:
        
        result = []
        
        d = m.groupdict()
        
        for key, value in d.items():
            
            result.append((entity_map[key.rsplit('_', maxsplit=1)[0].replace('_', '-')], value))
            
        return result
    
    return None

In [228]:
from tqdm import tqdm

result = []

for e in tqdm(v_12):
    
    for lexe in e.entry['lexes']:
        
        if lexe['template'] and lexe['template'] != 'NOT-FOUND':
        
            r = extract(lexe['ltext'], lexe['template'], e.entry['entity_map'])
            
            if r:

                result.extend(r)

100%|██████████| 7812/7812 [00:05<00:00, 1319.10it/s]


In [230]:
result[:10]

[('Aarhus_Airport', 'Aarhus Airport'),
 ('"Aarhus, Denmark"', 'Aarhus, Denmark'),
 ('Aarhus_Airport', 'Aarhus airport'),
 ('Aarhus', 'Aarhus'),
 ('Aarhus_Airport', 'Aarhus Airport'),
 ('25.0', '25 metres'),
 ('Aarhus_Airport', 'Aarhus airport'),
 ('25.0', '25 metres'),
 ('Aarhus_Airport', 'Aarhus Airport'),
 ('25.0', '25.0')]

In [232]:
s = list(set(result))

In [238]:
s[:10]

[('"hot"', 'hot'),
 ('Harcourt_(publisher)', 'Harcourt (publishers)'),
 ('African_Americans', 'D.C. African Americans'),
 ('Philippines', "The country's"),
 ('12.0', '12'),
 ('Olympic_Stadium_(Athens)', 'The Olympic Stadium (in Athens)'),
 ('HAL_Light_Combat_Helicopter', 'HAL light combat helicopters'),
 ('California_State_Assembly', 'the California State Assembly'),
 ('A.S._Gubbio_1910', 'the club'),
 ('"Deceased"', 'The')]

In [236]:
with open('felipe_melo.txt', 'w') as f:
    
    for r in s:
        f.write('{},{}\n'.format(*r))

In [237]:
!head felipe_melo.txt

"hot",hot
Harcourt_(publisher),Harcourt (publishers)
African_Americans,D.C. African Americans
Philippines,The country's
12.0,12
Olympic_Stadium_(Athens),The Olympic Stadium (in Athens)
HAL_Light_Combat_Helicopter,HAL light combat helicopters
California_State_Assembly,the California State Assembly
A.S._Gubbio_1910,the club
"Deceased",The
